In [3]:
import sys
sys.path.append("..")

%load_ext autoreload
%autoreload 2

In [9]:
!pip install transformers


     |████████████████████████████████| 3.2 MB 17.6 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4


In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
from lib.data_helper import create_full_data

from lib.helpers import split_by_doc_id, do_metrics, run_full_cv

In [12]:
from lib.models import IRTNet, IdealNet

# Only Style and Quality

In [13]:
style_data = create_full_data(feature_types=['style_quality_scaled'], label_type='bin_points')

In [14]:
style_data.head()

,feats,user_id,label,doc_id
0,"[1.849750499634872, 0.2499057065713443, -1.110...",395,False,.9999-repeating-1/1/Singularity
1,"[1.849750499634872, 0.2499057065713443, -1.110...",491,True,.9999-repeating-1/1/Singularity
2,"[1.849750499634872, 0.2499057065713443, -1.110...",140,True,.9999-repeating-1/1/Singularity
3,"[1.849750499634872, 0.2499057065713443, -1.110...",908,False,.9999-repeating-1/1/Singularity
4,"[1.849750499634872, 0.2499057065713443, -1.110...",600,True,.9999-repeating-1/1/Singularity


In [15]:
D_in = len(style_data.iloc[0].feats)
num_users = style_data.user_id.nunique()

In [16]:
model_arguments = {'D_in': D_in, 'num_users': num_users}
train_arguments = {'num_train_epochs': 200, 'learning_rate': 0.001}

In [17]:
model_cls = IRTNet

In [18]:
final_results = {}

In [19]:
for C in [1e-3, 1e-4, 1e-5, 1e-6]:
    for reg_type in ['l1', 'l2']:
        for learning_rate in [0.01, 0.005]:
            model_arguments['C'] = C
            model_arguments['reg_type'] = reg_type
            train_arguments['learning_rate'] = learning_rate
            train_arguments['num_train_epochs'] = 20
            results = run_full_cv(style_data, model_cls, model_arguments, train_arguments, averaged=True)
            
            final_results[(C, reg_type, learning_rate)] = results

{'loss': 0.7762, 'learning_rate': 0.009309773605742684, 'epoch': 1.38}
{'loss': 0.7696, 'learning_rate': 0.008619547211485367, 'epoch': 2.76}
{'loss': 0.7613, 'learning_rate': 0.007929320817228051, 'epoch': 4.14}
{'loss': 0.7553, 'learning_rate': 0.0072390944229707346, 'epoch': 5.52}
{'loss': 0.7484, 'learning_rate': 0.006548868028713418, 'epoch': 6.9}
{'loss': 0.7413, 'learning_rate': 0.005858641634456103, 'epoch': 8.28}
{'loss': 0.7341, 'learning_rate': 0.005168415240198785, 'epoch': 9.66}
{'loss': 0.7279, 'learning_rate': 0.004478188845941469, 'epoch': 11.04}
{'loss': 0.722, 'learning_rate': 0.0037879624516841525, 'epoch': 12.42}
{'loss': 0.7143, 'learning_rate': 0.003097736057426836, 'epoch': 13.8}
{'loss': 0.7085, 'learning_rate': 0.0024075096631695197, 'epoch': 15.18}
{'loss': 0.7014, 'learning_rate': 0.001717283268912203, 'epoch': 16.57}
{'loss': 0.6954, 'learning_rate': 0.0010270568746548869, 'epoch': 17.95}
{'loss': 0.6893, 'learning_rate': 0.0003368304803975704, 'epoch': 19.3

KeyboardInterrupt: 

In [35]:
for k, v in final_results.items():
    print(k, v['eval_accuracy'])

(0.001, 'l1', 0.01) 0.5556638545730683
(0.001, 'l1', 0.005) 0.5549213560644901
(0.001, 'l2', 0.01) 0.5495329919299549
(0.001, 'l2', 0.005) 0.5512404666327381
(0.0001, 'l1', 0.01) 0.5549180679829275
(0.0001, 'l1', 0.005) 0.5554921675236815
(0.0001, 'l2', 0.01) 0.5231370695640328
(0.0001, 'l2', 0.005) 0.5235921899302824
(1e-05, 'l1', 0.01) 0.5291729410379541
(1e-05, 'l1', 0.005) 0.5290745218895615
(1e-05, 'l2', 0.01) 0.5197905303229591
(1e-05, 'l2', 0.005) 0.5200414937686749
(1e-06, 'l1', 0.01) 0.5202582203221005
(1e-06, 'l1', 0.005) 0.5207080656018237
(1e-06, 'l2', 0.01) 0.5194316024941598
(1e-06, 'l2', 0.005) 0.5196631310601815


# Speaker Only

In [73]:
style_data = create_full_data(feature_types=['issues_speaker'], label_type='bin_points')

In [74]:
final_results = {}

In [75]:

D_in = len(style_data.iloc[0].feats)
num_users = style_data.user_id.nunique()
model_arguments = {'D_in': D_in, 'num_users': num_users, 'use_popularity': True}
model_cls = IdealNet

In [ ]:
train_arguments = {}
for C in [1e-4, 1e-5, 1e-6]:
    for reg_type in ['l1', 'l2']:
        for learning_rate in [0.1, 0.01, 0.005]:
            with open('log_file', 'a') as log_file:
                log_file.write(f"Starting {(C, reg_type, learning_rate)}\n")
            model_arguments['C'] = C
            model_arguments['reg_type'] = reg_type
            train_arguments['learning_rate'] = learning_rate
            train_arguments['num_train_epochs'] = 20
            results = run_full_cv(style_data, model_cls, model_arguments, train_arguments, averaged=True)
            
            final_results[(C, reg_type, learning_rate)] = results
            with open('log_file', 'a') as log_file:
                log_file.write(f"Results {str(results)}\n\n")
                

{'loss': 1.2819, 'learning_rate': 0.09314599040438658, 'epoch': 1.37}
{'loss': 1.2101, 'learning_rate': 0.08629198080877314, 'epoch': 2.74}
{'loss': 1.1442, 'learning_rate': 0.07943797121315971, 'epoch': 4.11}
{'loss': 1.0726, 'learning_rate': 0.07258396161754627, 'epoch': 5.48}
{'loss': 1.0313, 'learning_rate': 0.06572995202193284, 'epoch': 6.85}
{'loss': 0.9809, 'learning_rate': 0.0588759424263194, 'epoch': 8.22}
{'loss': 0.925, 'learning_rate': 0.05202193283070597, 'epoch': 9.6}
{'loss': 0.8872, 'learning_rate': 0.04516792323509253, 'epoch': 10.97}
{'loss': 0.8492, 'learning_rate': 0.0383139136394791, 'epoch': 12.34}
{'loss': 0.8121, 'learning_rate': 0.031459904043865665, 'epoch': 13.71}
{'loss': 0.7776, 'learning_rate': 0.02460589444825223, 'epoch': 15.08}
{'loss': 0.7489, 'learning_rate': 0.017751884852638795, 'epoch': 16.45}
{'loss': 0.7192, 'learning_rate': 0.01089787525702536, 'epoch': 17.82}
{'loss': 0.6923, 'learning_rate': 0.004043865661411926, 'epoch': 19.19}
{'train_runtim

{'loss': 0.6894, 'learning_rate': 0.006584232818691078, 'epoch': 6.83}
{'loss': 0.6866, 'learning_rate': 0.005901079382429294, 'epoch': 8.2}
{'loss': 0.6851, 'learning_rate': 0.005217925946167509, 'epoch': 9.56}
{'loss': 0.6797, 'learning_rate': 0.004534772509905725, 'epoch': 10.93}
{'loss': 0.6757, 'learning_rate': 0.0038516190736439405, 'epoch': 12.3}
{'loss': 0.6745, 'learning_rate': 0.0031684656373821563, 'epoch': 13.66}
{'loss': 0.6698, 'learning_rate': 0.0024853122011203716, 'epoch': 15.03}
{'loss': 0.6651, 'learning_rate': 0.0018021587648585872, 'epoch': 16.4}
{'loss': 0.6629, 'learning_rate': 0.001119005328596803, 'epoch': 17.76}
{'loss': 0.6568, 'learning_rate': 0.0004358518923350185, 'epoch': 19.13}
{'train_runtime': 430.866, 'train_samples_per_second': 339.734, 'epoch': 20.0}
Finished round {'eval_loss': 0.6680341362953186, 'eval_macro_f1': 0.6023363527760912, 'eval_micro_f1': 0.6093783303730018, 'eval_pos_f1': 0.5494181281757089, 'eval_accuracy': 0.6093783303730018, 'eval_r

{'loss': 0.6675, 'learning_rate': 0.0022485899023249414, 'epoch': 11.01}
{'loss': 0.6641, 'learning_rate': 0.0019046636401155593, 'epoch': 12.38}
{'loss': 0.6625, 'learning_rate': 0.001560737377906177, 'epoch': 13.76}
{'loss': 0.6605, 'learning_rate': 0.0012168111156967947, 'epoch': 15.13}
{'loss': 0.6571, 'learning_rate': 0.0008728848534874123, 'epoch': 16.51}
{'loss': 0.6553, 'learning_rate': 0.0005289585912780301, 'epoch': 17.88}
{'loss': 0.6524, 'learning_rate': 0.0001850323290686477, 'epoch': 19.26}
{'train_runtime': 429.3067, 'train_samples_per_second': 338.639, 'epoch': 20.0}
Finished round {'eval_loss': 0.6733263731002808, 'eval_macro_f1': 0.6088389719283904, 'eval_micro_f1': 0.6156025428413489, 'eval_pos_f1': 0.5574031346964755, 'eval_accuracy': 0.6156025428413489, 'eval_runtime': 2.1556, 'eval_samples_per_second': 6713.78}
{'loss': 0.6857, 'learning_rate': 0.00465407499654075, 'epoch': 1.38}
{'loss': 0.6799, 'learning_rate': 0.0043081499930815, 'epoch': 2.77}
{'loss': 0.6792,

{'loss': 0.7216, 'learning_rate': 0.016977999169779993, 'epoch': 16.6}
{'loss': 0.6964, 'learning_rate': 0.010059499100594992, 'epoch': 17.99}
{'loss': 0.6649, 'learning_rate': 0.0031409990314099907, 'epoch': 19.37}
{'train_runtime': 445.789, 'train_samples_per_second': 324.234, 'epoch': 20.0}
Finished round {'eval_loss': 0.6528710126876831, 'eval_macro_f1': 0.6187174371917529, 'eval_micro_f1': 0.625320729237002, 'eval_pos_f1': 0.5685405489464271, 'eval_accuracy': 0.625320729237002, 'eval_runtime': 2.3235, 'eval_samples_per_second': 6373.872}
{'loss': 1.2679, 'learning_rate': 0.0930584478689435, 'epoch': 1.39}
{'loss': 1.2118, 'learning_rate': 0.086116895737887, 'epoch': 2.78}
{'loss': 1.1526, 'learning_rate': 0.07917534360683048, 'epoch': 4.16}
{'loss': 1.0828, 'learning_rate': 0.07223379147577398, 'epoch': 5.55}
{'loss': 1.0289, 'learning_rate': 0.06529223934471749, 'epoch': 6.94}
{'loss': 0.9701, 'learning_rate': 0.05835068721366098, 'epoch': 8.33}
{'loss': 0.9132, 'learning_rate': 

{'train_runtime': 444.7286, 'train_samples_per_second': 323.928, 'epoch': 20.0}
Finished round {'eval_loss': 0.6830961108207703, 'eval_macro_f1': 0.5963158734020317, 'eval_micro_f1': 0.6027205441088218, 'eval_pos_f1': 0.5454684162343606, 'eval_accuracy': 0.6027205441088218, 'eval_runtime': 2.372, 'eval_samples_per_second': 6322.612}
{'loss': 0.6603, 'learning_rate': 0.004657299520219329, 'epoch': 1.37}
{'loss': 0.6478, 'learning_rate': 0.0043145990404386565, 'epoch': 2.74}
{'loss': 0.6462, 'learning_rate': 0.003971898560657985, 'epoch': 4.11}
{'loss': 0.6431, 'learning_rate': 0.0036291980808773132, 'epoch': 5.48}
{'loss': 0.6464, 'learning_rate': 0.003286497601096642, 'epoch': 6.85}
{'loss': 0.6441, 'learning_rate': 0.0029437971213159696, 'epoch': 8.22}
{'loss': 0.6441, 'learning_rate': 0.002601096641535298, 'epoch': 9.6}
{'loss': 0.6423, 'learning_rate': 0.0022583961617546264, 'epoch': 10.97}
{'loss': 0.6414, 'learning_rate': 0.001915695681973955, 'epoch': 12.34}
{'loss': 0.6421, 'lea

In [78]:
max(v['eval_accuracy'] for v in final_results.values())

0.6127768908055097

# Style Data and Speaker Issues

In [56]:
style_data = create_full_data(feature_types=['style_quality_scaled', 'issues_speaker'], label_type='bin_points')

In [63]:
len(style_data.iloc[0].feats)

181

In [57]:
final_results_v2 = {}

In [64]:

D_in = len(style_data.iloc[0].feats)
num_users = style_data.user_id.nunique()
model_arguments = {'D_in': D_in, 'num_users': num_users, 'use_popularity': True}


In [65]:
model_cls = IdealNet

In [61]:
train_arguments = {}
for C in [1e-4, 1e-5, 1e-6]:
    for reg_type in ['l1', 'l2']:
        for learning_rate in [0.1, 0.01, 0.005]:
            with open('log_file', 'a') as log_file:
                log_file.write(f"Starting {(C, reg_type, learning_rate)}\n")
            model_arguments['C'] = C
            model_arguments['reg_type'] = reg_type
            train_arguments['learning_rate'] = learning_rate
            train_arguments['num_train_epochs'] = 20
            results = run_full_cv(style_data, model_cls, model_arguments, train_arguments, averaged=True)
            
            final_results_v2[(C, reg_type, learning_rate)] = results
            with open('log_file', 'a') as log_file:
                log_file.write(f"Results {str(results)}\n\n")
                

{'train_runtime': 442.3947, 'train_samples_per_second': 329.796, 'epoch': 20.0}
Finished round {'eval_loss': 0.6900224089622498, 'eval_macro_f1': 0.6092049021999189, 'eval_micro_f1': 0.6092107107808776, 'eval_pos_f1': 0.6076982619097882, 'eval_accuracy': 0.6092107107808776, 'eval_runtime': 2.3229, 'eval_samples_per_second': 6141.387}
{'train_runtime': 443.634, 'train_samples_per_second': 329.957, 'epoch': 20.0}
Finished round {'eval_loss': 0.6941145658493042, 'eval_macro_f1': 0.6127616009191447, 'eval_micro_f1': 0.6128596802841918, 'eval_pos_f1': 0.6189244003077139, 'eval_accuracy': 0.6128596802841918, 'eval_runtime': 2.1969, 'eval_samples_per_second': 6406.648}


KeyboardInterrupt: 

In [ ]:
import pickle
pickle.dump(final_results_v2, open('debate_speaker_style.pkl', 'wb'))

In [72]:
final_results_v2

{}

In [71]:
model_arguments['C'] = 1e-4
model_arguments['reg_type'] = 'l1'
model_arguments['init_type'] = 'uniform'
train_arguments['learning_rate'] = 0.01
train_arguments['num_train_epochs'] = 25
results = run_full_cv(style_data, model_cls, model_arguments, train_arguments, averaged=True)

{'loss': 0.7875, 'learning_rate': 0.009451679232350926, 'epoch': 1.37}
{'loss': 0.7761, 'learning_rate': 0.00890335846470185, 'epoch': 2.74}
{'loss': 0.7689, 'learning_rate': 0.008355037697052776, 'epoch': 4.11}
{'loss': 0.7563, 'learning_rate': 0.007806716929403701, 'epoch': 5.48}
{'loss': 0.752, 'learning_rate': 0.0072583961617546265, 'epoch': 6.85}
{'loss': 0.7414, 'learning_rate': 0.006710075394105551, 'epoch': 8.22}
{'loss': 0.7318, 'learning_rate': 0.006161754626456477, 'epoch': 9.6}
{'loss': 0.7246, 'learning_rate': 0.005613433858807403, 'epoch': 10.97}
{'loss': 0.7152, 'learning_rate': 0.005065113091158328, 'epoch': 12.34}
{'loss': 0.71, 'learning_rate': 0.004516792323509253, 'epoch': 13.71}
{'loss': 0.6985, 'learning_rate': 0.003968471555860178, 'epoch': 15.08}
{'loss': 0.6913, 'learning_rate': 0.003420150788211104, 'epoch': 16.45}
{'loss': 0.6844, 'learning_rate': 0.002871830020562029, 'epoch': 17.82}
{'loss': 0.6753, 'learning_rate': 0.002323509252912954, 'epoch': 19.19}
{'l

### No popularity

In [31]:
D_in = len(style_data.iloc[0].feats)
num_users = style_data.user_id.nunique()
model_arguments = {'D_in': D_in, 'num_users': num_users, 'use_popularity': False}


In [42]:
log_file = open('log_file', 'a')

In [45]:
log_file.write(f"HI Starting {(C, reg_type, learning_rate)}\n")

32

In [47]:
!cat log_file

Starting (0.0001, 'l1', 0.1)
Starting (0.0001, 'l1', 0.1)
Starting (0.0001, 'l1', 0.1)
Starting (0.0001, 'l1', 0.1)
HI Starting (0.0001, 'l1', 0.1)


In [35]:
train_arguments = {}
for C in [1e-4, 1e-5, 1e-6]:
    for reg_type in ['l1', 'l2']:
        for learning_rate in [0.1, 0.01, 0.005]:
            
            with open('log_file', 'a') as log_file:
                log_file.write(f"Starting {(C, reg_type, learning_rate)}\n")
            model_arguments['C'] = C
            model_arguments['reg_type'] = reg_type
            train_arguments['learning_rate'] = learning_rate
            train_arguments['num_train_epochs'] = 20
            results = run_full_cv(style_data, model_cls, model_arguments, train_arguments, averaged=True)
            
            final_results_v2[(C, reg_type, learning_rate)] = results

KeyboardInterrupt: 

In [55]:
import pickle
pickle.dump(final_results_v2, open('debate_non_pop.pkl', 'wb'))

In [51]:
for k, v in final_results_v2.items():
    print(k, v['eval_accuracy'])

(0.0001, 'l1', 0.1) 0.6004917872773381
(0.0001, 'l1', 0.01) 0.6032701927099586
(0.0001, 'l1', 0.005) 0.6044170013286541
(0.0001, 'l2', 0.1) 0.6018610931401083
(0.0001, 'l2', 0.01) 0.60541814686578
(0.0001, 'l2', 0.005) 0.6063376070860166
(1e-05, 'l1', 0.1) 0.6004062185140656
(1e-05, 'l1', 0.01) 0.6056387457822618
(1e-05, 'l1', 0.005) 0.6059731815058973
(1e-05, 'l2', 0.1) 0.6010279632254549
(1e-05, 'l2', 0.01) 0.6047147257982896
(1e-05, 'l2', 0.005) 0.6060131567136049
(1e-06, 'l1', 0.1) 0.6003834619523063
(1e-06, 'l1', 0.01) 0.6051268005622147
(1e-06, 'l1', 0.005) 0.6062514589793431
(1e-06, 'l2', 0.1) 0.6006256011182876
(1e-06, 'l2', 0.01) 0.6054351251609228
(1e-06, 'l2', 0.005) 0.6060959422928347


In [53]:
max([v.get('eval_accuracy') for v in final_results_v2.values()])

0.6063376070860166

In [54]:
model_arguments

{'D_in': 181,
 'num_users': 1131,
 'use_popularity': False,
 'C': 1e-06,
 'reg_type': 'l2'}